In [ ]:
!pip install langchain -qU
!pip install langchain-openai -qU
!pip install langchain-community -qU

!pip install duckduckgo-search -qU
!pip install pyowm -qU

!pip install langchainhub -qU
!pip install google-api-python-client -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 114.6 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

## **Initialize OpenAI LLM**

In [ ]:
from langchain_openai import ChatOpenAI

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

## **Initialize DuckDuckGo Search Tool**

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

search_tool

DuckDuckGoSearchRun(api_wrapper=DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='api', source='text'))

## **Initialize OpenWeatherMap Tool**

In [ ]:
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from langchain.agents import load_tools

os.environ["OPENWEATHERMAP_API_KEY"] = userdata.get('OPENWEATHERMAP_API_KEY')

weather = OpenWeatherMapAPIWrapper()

weather_tool = load_tools(["openweathermap-api"], llm)[0]

weather_tool

OpenWeatherMapQueryRun(api_wrapper=OpenWeatherMapAPIWrapper(owm=<pyowm.owm.OWM - API key=************************bd5c55c8, subscription type=free, PyOWM version=(3, 3, 0)>, openweathermap_api_key=None))

## **Load Prompt Template from LangChainHub**

In [ ]:
from langchain import hub

prompt = hub.pull("hwchase17/react")

print(prompt)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


## **Create Built-in Agent**

In [ ]:
from langchain.agents import create_react_agent

# List of tools for the agent
tools = [search_tool, weather_tool]

# Create the agent using the LLM and the prompt template
agent = create_react_agent(tools = tools, llm = llm, prompt=prompt)

In [ ]:
from langchain.agents import AgentExecutor

# Initialize the agent executor with the created agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [ ]:

response = agent_executor.invoke({"input": "What's the weather in Kandy?"})

print(response['output'])



> Entering new AgentExecutor chain...
I should use the open_weather_map tool to get the current weather in Kandy.
Action: open_weather_map
Action Input: KandyIn Kandy, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 1.07 m/s, direction: 49°
Humidity: 82%
Temperature: 
  - Current: 25.27°C
  - High: 25.27°C
  - Low: 25.27°C
  - Feels like: 25.99°C
Rain: {}
Heat index: None
Cloud cover: 89%I have the current weather information for Kandy.
Final Answer: The current weather in Kandy is overcast clouds with a temperature of 25.27°C.

> Finished chain.
The current weather in Kandy is overcast clouds with a temperature of 25.27°C.


In [ ]:
response = agent_executor.invoke({"input": "what is machine learning?"})

print(response['output'])



> Entering new AgentExecutor chain...
I should search for a definition of machine learning to provide an accurate answer.
Action: duckduckgo_search
Action Input: machine learning definitionLearn what machine learning is, how it works, and the different types of it powering the services and applications we rely on every day. Explore real-world examples, benefits and dangers, and beginner-friendly courses on Coursera. Machine learning is a branch of artificial intelligence that enables algorithms to learn from data and make predictions without explicit programming. Learn the difference between machine learning and traditional programming, the types of machine learning, the applications, and the limitations. Machine learning is a type of artificial intelligence that allows computers to learn without being explicitly programmed. Learn the concept, classification, examples, benefits and challenges of machine learning in this article. Machine learning is a method of data analysis that auto

In [ ]:
response = agent_executor.invoke({"input": "how many words count within introduction to machine learning?"})

print(response['output'])



> Entering new AgentExecutor chain...
I should search for the introduction to machine learning to find out how many words it typically contains.
Action: duckduckgo_search
Action Input: introduction to machine learning word countBag of Words just creates a set of vectors containing the count of word occurrences in the document (reviews), while the TF-IDF model contains information on the more important words and the less important ones as well. Bag of Words vectors are easy to interpret. However, TF-IDF usually performs better in machine learning models. Reinforcement Learning is a branch of Machine Learning, also called Online Learning. It is used to decide what action to take at t+1 based on data up to time t. This concept is used in Artificial Intelligence applications such as walking. A popular example of reinforcement learning is a chess engine Bag of words (BoW; also stylized as bag-of-words) is a feature extraction technique that models text data for processing in information r

In [ ]:
from googleapiclient.discovery import build
from langchain.agents import tool

# Define the YouTube API tool
@tool
def youtube_search(query: str) -> str:
    """
    Search for YouTube videos based on a query.
    Returns a list of video titles and URLs.
    """
    api_key = userdata.get("YOUTUBE_API_KEY")
    youtube = build("youtube", "v3", developerKey=api_key)

    # Perform the search
    search_response = youtube.search().list(
        q=query,
        part="snippet",
        maxResults=5
    ).execute()

    # Extract and format results
    results = [
        f"{item['snippet']['title']} - https://www.youtube.com/watch?v={item['id']['videoId']}"
        for item in search_response.get("items", [])
        if item["id"]["kind"] == "youtube#video"
    ]
    return "\n".join(results) if results else "No results found."


In [ ]:
# Add the YouTube search tool to the tools list
tools.append(youtube_search)

# Create a new agent with the YouTube search tool
agent = create_react_agent(tools=tools, llm=llm, prompt=prompt)

# Initialize the agent executor with the new agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)


In [ ]:
from langchain.agents import tool

@tool
def word_count(text: str) -> int:
  """Returns the word count."""
  return len(text.split())

In [ ]:
# Add the custom tool to the list of tools
tools.append(word_count)

print(tools)

[DuckDuckGoSearchRun(api_wrapper=DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='api', source='text')), OpenWeatherMapQueryRun(api_wrapper=OpenWeatherMapAPIWrapper(owm=<pyowm.owm.OWM - API key=************************bd5c55c8, subscription type=free, PyOWM version=(3, 3, 0)>, openweathermap_api_key=None)), StructuredTool(name='youtube_search', description='Search for YouTube videos based on a query.\nReturns a list of video titles and URLs.', args_schema=<class 'langchain_core.utils.pydantic.youtube_search'>, func=<function youtube_search at 0x7c7424b9cdc0>), StructuredTool(name='word_count', description='Returns the word count.', args_schema=<class 'langchain_core.utils.pydantic.word_count'>, func=<function word_count at 0x7c744eb19c60>)]


In [ ]:
# Create a new agent with the DuckDuckGo search tool, OpenWeatherMap tool  and custom word count tool
agent = create_react_agent(tools = tools, llm = llm, prompt=prompt)

# Initialize the agent executor with the new agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [ ]:
response = agent_executor.invoke({"input": "what is machine learning?"})

print(response['output'])



> Entering new AgentExecutor chain...
I should search for a definition of machine learning to provide a clear explanation.
Action: duckduckgo_search
Action Input: "definition of machine learning"Learn what machine learning is, how it works, and the different types of it powering the services and applications we rely on every day. Explore real-world examples, benefits and dangers, and beginner-friendly courses on Coursera. Machine learning is a branch of artificial intelligence that enables algorithms to learn from data and make predictions without explicit programming. Learn the difference between machine learning and traditional programming, the types of machine learning, the applications, and the limitations. The definition of machine learning can be defined as that machine learning gives computers the ability to learn without being explicitly programmed. Also in 1997, Tom Mitchell defined machine learning that "A computer program is sa ... Machine learning is one of the most excit

In [ ]:
response = agent_executor.invoke({"input": "how many words count within broken angle song"})

print(response['output'])



> Entering new AgentExecutor chain...
I should use the word_count tool to count the number of words in the song "Broken Angle."
Action: word_count
Action Input: "Broken Angle song lyrics"4I now know the number of words in the song "Broken Angle."
Final Answer: 4 words

> Finished chain.
4 words


In [ ]:
response = agent_executor.invoke({"input": "what is the temperature of the city where 'lotus tower' is located?"})

print(response['output'])



> Entering new AgentExecutor chain...
I should use the open_weather_map tool to find the temperature of the city where the Lotus Tower is located.
Action: open_weather_map
Action Input: Colombo, Sri LankaIn Colombo, Sri Lanka, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 1.92 m/s, direction: 21°
Humidity: 78%
Temperature: 
  - Current: 28.97°C
  - High: 28.97°C
  - Low: 28.97°C
  - Feels like: 34.23°C
Rain: {}
Heat index: None
Cloud cover: 93%The temperature of the city where the Lotus Tower is located is 28.97°C.
Final Answer: 28.97°C

> Finished chain.
28.97°C


In [ ]:
response = agent_executor.invoke({"input": "machine learning sinhala "})
print(response['output'])



> Entering new AgentExecutor chain...
I should search for information on machine learning in Sinhala language.
Action: duckduckgo_search
Action Input: machine learning sinhalaTrain Test Split in Machine Learning | Sinhalaඅපි සාමාන්‍යයෙන් Machine Learning Model එකක් train කරන්න කලින් Data Set එක ... One Hot Encoding in Machine Learning | Sinhala. Machine learning භාවිතා කරලා ගැටලුවක් විසඳන්න යනකොට අපි කරන ප්‍රධානම කාර්යය වන්නේ දත්ත එකතු කර ගන්න එක. ඒ දත්ත process කරන්න අවශ්‍යයි. grade 12 grade 13 MCQ Sinhala medium English medium lesson note unit 1 unit 2 unit 3 A/L ICT 🧠 Machine Learning 🧠 Automation පැත්තෙන් ඉස්සරහට යන්න? ඒත් AI නිසා ඔයාට Chance එකක් ලැබෙන එකක් නෑ කියලා බයද? 😨 ... (UI/UX) for Beginners in Sinhala:Batch 01 by Shiran Sanjeewa Weerasinghe. November 29, 2024. Digital Marketing with ... 🧠 Machine Learning 🧠 Automation පැත්තෙන් ඉස්සරහට යන්න? ඒත් AI නිසා ඔයාට Chance එකක් ලැබෙන එකක් නෑ කියලා බයද? 😨 ... Web Designing for Beginners with AI in Sinhala by Sanjaya Elvitigala & N